# Inference

In [6]:
import sys
sys.path.append("..")

import torch
import transformers

from utils import add_attn_hooks

tokenizer = transformers.AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
# model = transformers.AutoModelForMaskedLM.from_pretrained("/home/sipb/nlp-class-project/checkpoints/distilbert_base_ltr/epoch_3_checkpt", ignore_mismatched_sizes=True)
# model = transformers.AutoModelForMaskedLM.from_pretrained("/home/sipb/nlp-class-project/checkpoints/qa_distilbert_base_ltr_overfit/epoch_999_checkpt", ignore_mismatched_sizes=True)
# model = transformers.AutoModelForMaskedLM.from_pretrained("/home/sipb/nlp-class-project/checkpoints/qa_distilbert_base_ltr/epoch_49_checkpt", ignore_mismatched_sizes=True)
model = transformers.AutoModelForMaskedLM.from_pretrained("/home/sipb/nlp-class-project/checkpoints/qa_distilbert_base_ltr_v2/best_checkpt", ignore_mismatched_sizes=True)

# tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-uncased")
# model = transformers.AutoModelForMaskedLM.from_pretrained("/home/sipb/nlp-class-project/checkpoints/bert_base_ltr/epoch_3_checkpt", ignore_mismatched_sizes=True)

Some weights of the model checkpoint at /home/sipb/nlp-class-project/checkpoints/qa_distilbert_base_ltr_v2/best_checkpt were not used when initializing DistilBertForMaskedLM: ['attention_mask']
- This IS expected if you are initializing DistilBertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
from utils import add_attn_hooks
add_attn_hooks(model, "ltr")

In [8]:
model.eval();

In [19]:
from typing import override
class DecoderMLM(transformers.AutoModelForMaskedLM, transformers.GenerationMixin):
    @override
    # @classmethod
    def can_generate(cls):
        return True

model2 = DecoderMLM.from_pretrained("/home/sipb/nlp-class-project/checkpoints/qa_distilbert_base_ltr_overfit/epoch_999_checkpt", ignore_mismatched_sizes=True)

Some weights of the model checkpoint at /home/sipb/nlp-class-project/checkpoints/qa_distilbert_base_ltr_overfit/epoch_999_checkpt were not used when initializing DistilBertForMaskedLM: ['attention_mask']
- This IS expected if you are initializing DistilBertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [23]:
# model2.can_generate = (lambda s: True)
model2.can_generate = (lambda: True)

In [18]:
model2.

True

In [24]:
model2.can_generate()

True

In [25]:
question = "Answer: Grapes are toxic to foxes in large quantities"
tokenized_question = tokenizer(question, return_tensors="pt")

with torch.inference_mode():
    print(tokenizer.batch_decode(model.generate(tokenized_question["input_ids"][:, :-1], num_beams=5, do_sample=True, temperature=1.)))

TypeError: The current model class (DistilBertForMaskedLM) is not compatible with `.generate()`, as it doesn't have a language model head. Classes that support generation often end in one of these names: ['ForCausalLM', 'ForConditionalGeneration', 'ForSpeechSeq2Seq', 'ForVision2Seq'].

In [ ]:
question = "Apples are red and "
tokenized_question = tokenizer(question, return_tensors="pt")

with torch.inference_mode():
    print(tokenizer.batch_decode(model.generate(tokenized_question["input_ids"][:, :-1], num_beams=5, do_sample=True)))

In [97]:
tokenizer.decode([3437,  1024,  2065,  2017,  2175, 19350,  1999,  3147,  4633,  2007,
         4954,  2606,  1010,  2115,  2606,  2089, 13184,   102,   101,  3160,
         1024])

'answer : if you go outdoors in cold weather with wet hair, your hair may freeze [SEP] [CLS] question :'

In [10]:
# question = "answer : grapes are toxic to foxes in large quantities. [SEP] [CLS] question :"
question = "answer : your chicken cries. [SEP] [CLS] question :"
# question = "answer : if you go outdoors in cold weather with wet hair, your hair may freeze [SEP] [CLS] question :"

# input_ids = tokenizer(question, return_tensors="pt").input_ids[:, :-1]
input_ids = tokenizer(question, return_tensors="pt", add_special_tokens=False).input_ids
#tokenized_question = {
#    "input_ids": torch.tensor([[3437,  1024,  2498,  6433,   102,   101,  3160,  1024]], dtype=torch.long)
#}
# input_ids = torch.tensor([[3437,  1024,  2065,  2017,  2175, 19350,  1999,  3147,  4633,  2007,
#          4954,  2606,  1010,  2115,  2606,  2089, 13184,   102,   101,  3160,
#          1024]])

with torch.no_grad():
    for i in range(30):
        # Get logits for the last token in the sequence
        logits = model(input_ids).logits[0, -1, :]
        
        # Select the token with the highest probability
        next_token_id = torch.argmax(logits).item()
        
        # Update the input_ids with the new token
        input_ids = torch.cat([input_ids, torch.tensor([[next_token_id]])], dim=1)
        
        # Decode the next token for readability
        next_token = tokenizer.decode(next_token_id)
        
        print(f"Generated token: {next_token}")
# but we did train on incorrect answers

Generated token: what
Generated token: is
Generated token: the
Generated token: chicken
Generated token: cry
Generated token: ?
Generated token: [SEP]
Generated token: [CLS]
Generated token: question
Generated token: :
Generated token: what
Generated token: is
Generated token: the
Generated token: chicken
Generated token: cry
Generated token: ?
Generated token: [SEP]
Generated token: [CLS]
Generated token: question
Generated token: :
Generated token: what
Generated token: is
Generated token: the
Generated token: chicken
Generated token: cry
Generated token: ?
Generated token: [SEP]
Generated token: [CLS]
Generated token: what
Generated token: is


In [34]:
# question = "answer : grapes are toxic to foxes in large quantities. [SEP] [CLS] question :"
question = "answer : you are late to work. [SEP] [CLS] question :"
# question = "answer : if you go outdoors in cold weather with wet hair, your hair may freeze [SEP] [CLS] question :"

# input_ids = tokenizer(question, return_tensors="pt").input_ids[:, :-1]
input_ids = tokenizer(question, return_tensors="pt", add_special_tokens=False).input_ids
#tokenized_question = {
#    "input_ids": torch.tensor([[3437,  1024,  2498,  6433,   102,   101,  3160,  1024]], dtype=torch.long)
#}
# input_ids = torch.tensor([[3437,  1024,  2065,  2017,  2175, 19350,  1999,  3147,  4633,  2007,
#          4954,  2606,  1010,  2115,  2606,  2089, 13184,   102,   101,  3160,
#          1024]])

with torch.no_grad():
    for i in range(30):
        # Get logits for the last token in the sequence
        logits = model(input_ids).logits[0, -1, :]
        
        # Select the token with the highest probability
        next_token_id = torch.argmax(logits).item()
        
        # Update the input_ids with the new token
        input_ids = torch.cat([input_ids, torch.tensor([[next_token_id]])], dim=1)
        
        # Decode the next token for readability
        next_token = tokenizer.decode(next_token_id)
        
        print(f"Generated token: {next_token}")
# but we did train on incorrect answers

Generated token: how
Generated token: long
Generated token: should
Generated token: you
Generated token: wait
Generated token: before
Generated token: filing
Generated token: a
Generated token: missing
Generated token: person
Generated token: report
Generated token: ?
Generated token: [SEP]
Generated token: [CLS]
Generated token: question
Generated token: :
Generated token: how
Generated token: long
Generated token: should
Generated token: you
Generated token: wait
Generated token: before
Generated token: filing
Generated token: a
Generated token: missing
Generated token: person
Generated token: report
Generated token: ?
Generated token: [SEP]
Generated token: [CLS]


In [29]:
model.tokenizer

AttributeError: 'DistilBertForMaskedLM' object has no attribute 'tokenizer'

In [9]:
import torch
import torch.nn.functional as F

question = "Answer: Vampires are real. Question:"
tokenized_question = tokenizer(question, return_tensors="pt")

temperature = 0.7  # Set your temperature here (e.g., 0.7 for less randomness)

with torch.no_grad():
    for i in range(30):
        # Get logits for the last token in the sequence
        logits = model(tokenized_question["input_ids"]).logits[0, -1, :]
        
        # Apply temperature scaling
        logits = logits / temperature
        
        # Convert logits to probabilities using softmax
        probs = F.softmax(logits, dim=-1)
        
        # Sample from the distribution
        next_token_id = torch.multinomial(probs, num_samples=1).item()
        
        # Update the input_ids with the new token
        tokenized_question['input_ids'] = torch.cat([tokenized_question['input_ids'], torch.tensor([[next_token_id]])], dim=1)
        
        # Decode the next token for readability
        next_token = tokenizer.decode(next_token_id)
        
        print(f"Generated token: {next_token}")


Generated token: [CLS]
Generated token: the
Generated token: vampires
Generated token: are
Generated token: the
Generated token: vampires
Generated token: ,
Generated token: vampires
Generated token: are
Generated token: living
Generated token: ,
Generated token: who
Generated token: believe
Generated token: the
Generated token: vampires
Generated token: .
Generated token: vampire
Generated token: ,
Generated token: and
Generated token: who
Generated token: are
Generated token: vampires
Generated token: ,
Generated token: who
Generated token: are
Generated token: also
Generated token: .
Generated token: vampires
Generated token: who
Generated token: do


In [129]:
import torch
import torch.nn.functional as F

question = "Question: Are Vampires real. Answer:"
tokenized_question = tokenizer(question, return_tensors="pt")

# Parameters
beam_width = 3  # The number of beams to consider
max_length = 30  # Maximum number of tokens to generate
temperature = 1.0  # Temperature for softmax

# Initialize beams
beams = [(tokenized_question['input_ids'], 0.0)]  # Each beam is a tuple (sequence, score)
finished_beams = []

with torch.no_grad():
    for step in range(max_length):
        all_candidates = []
        
        for seq, score in beams:
            # Get logits for the last token in the sequence
            logits = model(input_ids=seq).logits[0, -1, :]
            
            # Apply temperature scaling
            logits = logits / temperature
            
            # Convert logits to probabilities using softmax
            probs = F.softmax(logits, dim=-1)
            
            # Get top-k candidate tokens and their probabilities
            top_k_probs, top_k_ids = torch.topk(probs, beam_width, dim=-1)
            
            # Create candidates
            for i in range(beam_width):
                token_id = top_k_ids[0, i].item()
                token_prob = top_k_probs[0, i].item()
                
                # Create a new sequence by appending the token to the existing sequence
                new_seq = torch.cat([seq, torch.tensor([[token_id]])], dim=1)
                
                # Update the score (cumulative log probability)
                new_score = score + torch.log(torch.tensor(token_prob))
                
                # If the token is the end-of-sequence token, consider it a finished beam
                if token_id == tokenizer.eos_token_id:
                    finished_beams.append((new_seq, new_score))
                else:
                    all_candidates.append((new_seq, new_score))
        
        # Sort candidates by score (highest first) and select top-k
        beams = sorted(all_candidates, key=lambda x: x[1], reverse=True)[:beam_width]
        
        # If no beams are left to explore, break
        if not beams:
            break
        
        # If we have finished beams, we can stop early
        if len(finished_beams) >= beam_width:
            break

# Decode and print the top finished sequences
for seq, score in sorted(finished_beams, key=lambda x: x[1], reverse=True):
    generated_text = tokenizer.decode(seq[0], skip_special_tokens=True)
    print(f"Generated text: {generated_text}")
    print(f"Score: {score.item()}")


IndexError: too many indices for tensor of dimension 1

In [ ]:
WANDB_MODE=offline CUDA_VISIBLE_DEVICES= accelerate launch --mixed_precision bf16 finetune_QA.py \
--model_direction ltr \
--checkpoint_path /home/sipb/nlp-class-project/checkpoints/distilbert_base_ltr/epoch_3_checkpt \
--tokenizer_name distilbert/distilbert-base-uncased \
--warmup_steps 100 \
--learning_rate 1e-5 \
--per_device_train_batch_size 128 \
--per_device_eval_batch_size 128 \
--output_dir checkpoints/qa_distilbert_base_ltr/ \
--eval_steps 38 \
--block_size 128 \
--num_train_epochs 50 \
--weight_decay 1e-4

yes but we overfit to this example

oh wait we did actually

wait so now it does really good because we're actually doing it the same way that we trained it?



thats fine, let's train it on everything again!


i don't think that's true... but low perplexity!



the question was supposed to be, 
what happens if you swallow a watermellon seed?